In [33]:
#pulling in dependents
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [34]:
#pulling in csv data for game reviews
csv_file = "resources/steam_reviews.csv"
reviews_df = pd.read_csv(csv_file)

#display dataframe
reviews_df.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title
0,2019-02-10,2,4,578,False,Recommended,&gt Played as German Reich&gt Declare war on B...,Expansion - Hearts of Iron IV: Man the Guns
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight


In [35]:
#convert title csv to dataframe
titles_df = pd.read_csv("resources/titles.csv")

titles_df.head()

,id,game_title
0,0,ACE COMBAT 7: SKIES UNKNOWN
1,1,ARK: Survival Evolved
2,2,ASTRONEER
3,3,Battlefleet Gothic: Armada 2
4,4,Beat Saber


In [36]:
#drop columns to leave game title, game review, recommendation, hours played, and date posted
dropped_reviews = reviews_df[['title','review','recommendation','hour_played','date_posted']]

#display dataframe
dropped_reviews.head()

,title,review,recommendation,hour_played,date_posted
0,Expansion - Hearts of Iron IV: Man the Guns,&gt Played as German Reich&gt Declare war on B...,Recommended,578,2019-02-10
1,Expansion - Hearts of Iron IV: Man the Guns,yes.,Recommended,184,2019-02-10
2,Expansion - Hearts of Iron IV: Man the Guns,Very good game although a bit overpriced in my...,Recommended,892,2019-02-07
3,Dead by Daylight,Out of all the reviews I wrote This one is pro...,Recommended,676,2018-06-14
4,Dead by Daylight,Disclaimer I survivor main. I play games for f...,Recommended,612,2017-06-20


In [37]:
#rename columns to match database in SQL
cleaned_reviews = dropped_reviews.rename(columns={'title': 'game_title'})

#display dataframe
cleaned_reviews.head()

,game_title,review,recommendation,hour_played,date_posted
0,Expansion - Hearts of Iron IV: Man the Guns,&gt Played as German Reich&gt Declare war on B...,Recommended,578,2019-02-10
1,Expansion - Hearts of Iron IV: Man the Guns,yes.,Recommended,184,2019-02-10
2,Expansion - Hearts of Iron IV: Man the Guns,Very good game although a bit overpriced in my...,Recommended,892,2019-02-07
3,Dead by Daylight,Out of all the reviews I wrote This one is pro...,Recommended,676,2018-06-14
4,Dead by Daylight,Disclaimer I survivor main. I play games for f...,Recommended,612,2017-06-20


In [38]:
#merge the game title and reviews with inner join on game_title column
merged_df = cleaned_reviews.merge(titles_df, on = "game_title")

merged_df.head()

,game_title,review,recommendation,hour_played,date_posted,id
0,Dead by Daylight,Out of all the reviews I wrote This one is pro...,Recommended,676,2018-06-14,6
1,Dead by Daylight,Disclaimer I survivor main. I play games for f...,Recommended,612,2017-06-20,6
2,Dead by Daylight,ENGLISH After playing for more than two years ...,Recommended,2694,2016-12-12,6
3,Dead by Daylight,Out of all the reviews I wrote This one is pro...,Recommended,48,2017-09-17,6
4,Dead by Daylight,I have never been told to kill myself more tha...,Recommended,71,2018-12-24,6


In [39]:
#replace the special characters in game title
merged_df.replace("ACE COMBATâ„¢ 7: SKIES UNKNOWN","ACE COMBAT 7: SKIES UNKNOWN")
merged_df.replace("Rocket LeagueÂ®", "Rocket League")
merged_df.replace("Sid Meier’s Civilization® VI", "Sid Meier's Civilization VI")
merged_df.replace("Sid Meierâ€™s Civilization VI", "Sid Meier's Civilization VI")
merged_df.replace("Tom Clancy's Rainbow Six Siege", "Tom Clancy's Rainbow Six Siege")

dictionary = {"®":'', "™":''}

merged_df.replace(dictionary, regex=True, inplace=True)

In [40]:
#count the dataframe; reviews column is missing some data -> users not leaving a review
merged_df.count()

game_title        366430
review            365056
recommendation    366430
hour_played       366430
date_posted       366430
id                366430
dtype: int64

In [41]:
#drop missing reviews  and count again
drop_reviews = merged_df.dropna()

drop_reviews.count()

game_title        365056
review            365056
recommendation    365056
hour_played       365056
date_posted       365056
id                365056
dtype: int64

In [42]:
#rearrage columns with id first
drop_reviews = drop_reviews[['id','game_title', 'review','recommendation','hour_played','date_posted']]

#rename id column to game_id
final_review = drop_reviews.rename(columns={'id': 'game_id'})

final_review.head()

,game_id,game_title,review,recommendation,hour_played,date_posted
0,6,Dead by Daylight,Out of all the reviews I wrote This one is pro...,Recommended,676,2018-06-14
1,6,Dead by Daylight,Disclaimer I survivor main. I play games for f...,Recommended,612,2017-06-20
2,6,Dead by Daylight,ENGLISH After playing for more than two years ...,Recommended,2694,2016-12-12
3,6,Dead by Daylight,Out of all the reviews I wrote This one is pro...,Recommended,48,2017-09-17
4,6,Dead by Daylight,I have never been told to kill myself more tha...,Recommended,71,2018-12-24


In [43]:
#add an id column 
final_review.reset_index(inplace=True)
final_review = final_review.rename(columns={"index":"id"})

final_review.head()

,id,game_id,game_title,review,recommendation,hour_played,date_posted
0,0,6,Dead by Daylight,Out of all the reviews I wrote This one is pro...,Recommended,676,2018-06-14
1,1,6,Dead by Daylight,Disclaimer I survivor main. I play games for f...,Recommended,612,2017-06-20
2,2,6,Dead by Daylight,ENGLISH After playing for more than two years ...,Recommended,2694,2016-12-12
3,3,6,Dead by Daylight,Out of all the reviews I wrote This one is pro...,Recommended,48,2017-09-17
4,4,6,Dead by Daylight,I have never been told to kill myself more tha...,Recommended,71,2018-12-24


In [44]:
#export table to csv
final_review.to_csv(r'review_table.csv', index = False)

In [45]:
#connect to SQL Postgres Database
rds_connection_string = "postgres:ChangeMe@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [46]:
#verifying table names in SQL Database
engine.table_names()

['game_table', 'review_table', 'user_table', 'purchase_table']

In [47]:
#upload file to SQL Database
final_review.to_sql(name='review_table', con=engine, if_exists='append', index=False)